# Chromatic progressions between bass notes

In [1]:
import os
import ms3
import pandas as pd
from helpers import cnt
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

In [2]:
CORPUS_PATH = "~/all_subcorpora/couperin_concerts"
RESULTS_PATH = os.path.abspath(os.path.join("..", "results"))

In [3]:
corpus_obj = ms3.Corpus(CORPUS_PATH)
corpus_obj.parse_tsv()
corpus_obj

[default|all]
Corpus 'couperin_concerts'
--------------------------
Location: C:\Users\hentsche/all_subcorpora/couperin_concerts
View: This view is called 'default'. It 
	- excludes pieces that are not contained in the metadata,
	- filters out file extensions requiring conversion (such as .xml), and
	- excludes review files and folders.

All 91 pieces are listed in 'metadata.tsv':

                               scores measures           notes        expanded       
                             detected detected parsed detected parsed detected parsed
c01n01_prelude                      1        1      1        1      1        1      1
c01n02_allemande                    1        1      1        1      1        1      1
c01n03_sarabande                    1        1      1        1      1        1      1
c01n04_gavotte                      1        1      1        1      1        1      1
c01n05_gigue                        1        1      1        1      1        1      1
c01n06_menuet

In [4]:
labels = corpus_obj.expanded()
labels

mc  mn quarterbeats  duration_qb mc_onset mn_onset  \
piece          i                                                         
c01n01_prelude 0     1   0            0         2.00        0      1/2   
               1     2   1            2         2.00        0        0   
               2     2   1            4         0.50      1/2      1/2   
               3     2   1          9/2         0.50      5/8      5/8   
               4     2   1            5         0.75      3/4      3/4   
...                 ..  ..          ...          ...      ...      ...   
parnasse_07    219  52  52        411/2         0.25      3/8      3/8   
               220  52  52        823/4         0.25     7/16     7/16   
               221  52  52          206         1.00      1/2      1/2   
               222  52  52          207         1.00      3/4      3/4   
               223  53  53          208         4.00        0        0   

                   timesig  staff  voice  label globalkey localkey pedal  \
piece          i                                                           
c01n01_prelude 0       4/4      1      1    G.I         G        I   NaN   
               1       4/4      1      1      V         G        I   NaN   
               2       4/4      1      1     I6         G        I   NaN   
               3       4/4      1      1      I         G        I   NaN   
               4       4/4      1      1   V(4)         G        I   NaN   
...                    ...    ...    ...    ...       ...      ...   ...   
parnasse_07    219     4/4      1      1    i64         b        i   NaN   
               220     4/4      1      1  iio64         b        i   NaN   
               221     4/4      1      1     i6         b        i   NaN   
               222     4/4      1      1      V         b        i   NaN   
               223     4/4      1      1    i\\         b        i   NaN   

                    chord numeral form figbass changes relativeroot cadence  \
piece          i                                                              
c01n01_prelude 0        I       I  NaN     NaN     NaN          NaN     NaN   
               1        V       V  NaN     NaN     NaN          NaN     NaN   
               2       I6       I  NaN       6     NaN          NaN     NaN   
               3        I       I  NaN     NaN     NaN          NaN     NaN   
               4     V(4)       V  NaN     NaN       4          NaN     NaN   
...                   ...     ...  ...     ...     ...          ...     ...   
parnasse_07    219    i64       i  NaN      64     NaN          NaN     NaN   
               220  iio64      ii    o      64     NaN          NaN     NaN   
               221     i6       i  NaN       6     NaN          NaN     NaN   
               222      V       V  NaN     NaN     NaN          NaN     NaN   
               223      i       i  NaN     NaN     NaN          NaN     NaN   

                   phraseend chord_type  globalkey_is_minor  \
piece          i                                              
c01n01_prelude 0         NaN          M               False   
               1         NaN          M               False   
               2         NaN          M               False   
               3         NaN          M               False   
               4         NaN          M               False   
...                      ...        ...                 ...   
parnasse_07    219       NaN          m                True   
               220       NaN          o                True   
               221       NaN          m                True   
               222       NaN          M                True   
               223        \\          m                True   

                    localkey_is_minor  chord_tones added_tones  root  \
piece          i                                                       
c01n01_prelude 0                False    (0, 4, 1)          ()     0   
    

## Transform `bass_note` column

### Expressing all bass notes as scale scale degrees of global tonic
Since all scale degrees are expressed as fifths-intervals, this is as easy as adding the local key expressed as fifths

In [5]:
transpose_by = ms3.transform(labels, ms3.roman_numeral2fifths, ['localkey', 'globalkey_is_minor'])
bass = labels.bass_note + transpose_by
bass.head()

piece           i
c01n01_prelude  0    0
                1    1
                2    4
                3    0
                4    1
dtype: Int64

#### Adding bass note names to DataFrame

In [6]:
transpose_by = ms3.transform(labels, ms3.name2fifths, ['globalkey'])
labels['bass_name'] = ms3.fifths2name(bass + transpose_by).values
labels.head()

mc  mn quarterbeats  duration_qb mc_onset mn_onset timesig  \
piece          i                                                               
c01n01_prelude 0   1   0            0         2.00        0      1/2     4/4   
               1   2   1            2         2.00        0        0     4/4   
               2   2   1            4         0.50      1/2      1/2     4/4   
               3   2   1          9/2         0.50      5/8      5/8     4/4   
               4   2   1            5         0.75      3/4      3/4     4/4   

                  staff  voice label globalkey localkey pedal chord numeral  \
piece          i                                                              
c01n01_prelude 0      1      1   G.I         G        I   NaN     I       I   
               1      1      1     V         G        I   NaN     V       V   
               2      1      1    I6         G        I   NaN    I6       I   
               3      1      1     I         G        I   NaN     I       I   
               4      1      1  V(4)         G        I   NaN  V(4)       V   

                 form figbass changes relativeroot cadence phraseend  \
piece          i                                                       
c01n01_prelude 0  NaN     NaN     NaN          NaN     NaN       NaN   
               1  NaN     NaN     NaN          NaN     NaN       NaN   
               2  NaN       6     NaN          NaN     NaN       NaN   
               3  NaN     NaN     NaN          NaN     NaN       NaN   
               4  NaN     NaN       4          NaN     NaN       NaN   

                 chord_type  globalkey_is_minor  localkey_is_minor  \
piece          i                                                     
c01n01_prelude 0          M               False              False   
               1          M               False              False   
               2          M               False              False   
               3          M               False              False   
               4          M               False              False   

                 chord_tones added_tones  root  bass_note  volta alt_label  \
piece          i                                                             
c01n01_prelude 0   (0, 4, 1)          ()     0          0   <NA>       NaN   
               1   (1, 5, 2)          ()     1          1   <NA>       NaN   
               2   (4, 1, 0)          ()     0          4   <NA>       NaN   
               3   (0, 4, 1)          ()     0          0   <NA>       NaN   
               4   (1, 0, 2)          ()     1          1   <NA>       NaN   

                 bass_name  
piece          i            
c01n01_prelude 0         G  
               1         D  
               2         B  
               3         G  
               4         D

#### Calculating intervals between successive bass notes
Sloppy version: Include intervals across movement boundaries

##### Bass progressions expressed in fifths

In [7]:
bass = bass.bfill()
ivs = bass - bass.shift()
ivs.value_counts()

-1    1499
0     1167
2      858
-2     835
3      802
-5     738
1      657
5      603
-4     565
4      307
-3     235
6       55
8       52
7       42
-8      20
-7      15
9       11
-6       9
-9       2
Name: count, dtype: Int64

##### Bass progressions expressed in (enharmonic) semitones

In [8]:
pc_ivs = ms3.fifths2pc(ivs)
pc_ivs.index = ivs.index
pc_ivs = pc_ivs.where(pc_ivs <= 6, pc_ivs % -6).fillna(0)
pc_ivs.value_counts()

 5    1499
 0    1168
 2     858
-2     835
-3     804
 1     780
-5     657
-1     618
-4     617
 4     327
 3     246
 6      64
Name: count, dtype: int64

### Chromatic bass progressions

#### Successive descending semitones

In [9]:
desc = cnt(pc_ivs, -1)
desc.n.value_counts()

n
1    588
2      7
3      4
4      1
Name: count, dtype: int64

##### Storing those with three or more

In [10]:
three_desc = labels.loc[desc[desc.n > 2].ixs.sum()]
three_desc.to_csv(os.path.join(RESULTS_PATH, 'three_desc.tsv'), sep='\t')
three_desc.head(30)

mc  mn quarterbeats  duration_qb mc_onset mn_onset  \
piece             i                                                        
c02n01_prelude    23  13  11           35         2.00      1/4      1/4   
                  24  14  12           37         2.00        0        0   
                  25  14  12           39         1.00      1/2      1/2   
                  26  15  13           40         1.00        0        0   
                  27  15  13           41         1.00      1/4      1/4   
                  53  25  23           70         2.00        0        0   
                  54  25  23           72         1.00      1/2      1/2   
                  55  26  24           73         2.00        0        0   
                  56  26  24           75         1.00      1/2      1/2   
                  57  27  25           76         1.00        0        0   
                  58  27  25           77         1.00      1/4      1/4   
c02n03_air_tendre 34  17  17           48         2.00        0        0   
                  35  17  17           50         1.00      1/2      1/2   
                  36  18  18           51         1.00        0        0   
                  37  18  18           52         1.00      1/4      1/4   
                  38  18  18           53         1.00      1/2      1/2   
                  39  19  19           54         1.00        0        0   
c05n02_allemande  37   8   7           30         0.50      7/8      7/8   
                  38   9   8         61/2         2.00        0        0   
                  39   9   8         65/2         1.00      1/2      1/2   
                  40   9   8         67/2         1.00      3/4      3/4   
                  41  10   9         69/2         1.00        0        0   
                  42  10   9         71/2         1.00      1/4      1/4   
c09n07_douceur    86  39  37           56         0.25     5/16     5/16   
                  87  40  38        225/4         1.00        0        0   
                  88  40  38        229/4         0.50      1/4      1/4   
                  89  41  39        231/4         1.50        0        0   
                  90  42  40        237/4         0.50        0        0   

                     timesig  staff  voice   label globalkey localkey pedal  \
piece             i                                                           
c02n01_prelude    23     3/4      1      1   V.I\\         D        V   NaN   
                  24     3/4      1      1      V6         D        V   NaN   
                  25     3/4      1      1  iii%43         D        V   NaN   
                  26     3/4      1      1  IV6(2)         D        V   NaN   
                  27     3/4      1      1     IV6         D        V   NaN   
                  53     3/4      1      1     I\\         D        I   NaN   
                  54     3/4      1      1     vi6         D        I   NaN   
                  55     3/4      1      1   vii%7         D        I   NaN   
                  56     3/4      1      1  iii%43         D        I   NaN   
                  57     3/4      1      1     vi7         D        I   NaN   
                  58     3/4      1      1     IV6         D        I   NaN   
c02n03_air_tendre 34     3/4      1      1   III.I         d      III   NaN   
                  35     3/4      1      1    V2/V         d      III   NaN   
                  36     3/4      1      1      V6         d      III   NaN   
                  37     3/4      1      1   IV/IV         d      III   NaN   
                  38     3/4      1      1   V2/IV         d      III   NaN   
                  39     3/4      1      1     IV6         d      III   NaN   
c05n02_allemande  37     2/2      1      1       I         F        V   NaN   
                  38     2/2      1      1      V6         F        V   NaN   
                  39     2/2      1      1      v6         F        V   NaN   
   

##### Storing those with four or more

In [11]:
four_desc = labels.loc[desc[desc.n > 3].ixs.sum()]
four_desc.to_csv(os.path.join(RESULTS_PATH, 'four_desc.tsv'), sep='\t')
four_desc.head(30)

mc  mn quarterbeats  duration_qb mc_onset mn_onset timesig  \
piece          i                                                                
c09n07_douceur 86  39  37           56         0.25     5/16     5/16     3/8   
               87  40  38        225/4         1.00        0        0     3/8   
               88  40  38        229/4         0.50      1/4      1/4     3/8   
               89  41  39        231/4         1.50        0        0     3/8   
               90  42  40        237/4         0.50        0        0     3/8   

                   staff  voice  label globalkey localkey pedal  chord  \
piece          i                                                         
c09n07_douceur 86      1      1      I         e        i   NaN      I   
               87      1      1     V6         e        i   NaN     V6   
               88      1      1     v6         e        i   NaN     v6   
               89      1      1  #vi%7         e        i   NaN  #vi%7   
               90      1      1  ii%43         e        i   NaN  ii%43   

                  numeral form figbass changes relativeroot cadence phraseend  \
piece          i                                                                
c09n07_douceur 86       I  NaN     NaN     NaN          NaN     NaN       NaN   
               87       V  NaN       6     NaN          NaN     NaN       NaN   
               88       v  NaN       6     NaN          NaN     NaN       NaN   
               89     #vi    %       7     NaN          NaN     NaN       NaN   
               90      ii    %      43     NaN          NaN     NaN       NaN   

                  chord_type  globalkey_is_minor  localkey_is_minor  \
piece          i                                                      
c09n07_douceur 86          M                True               True   
               87          M                True               True   
               88          m                True               True   
               89         %7                True               True   
               90         %7                True               True   

                      chord_tones added_tones  root  bass_note  volta  \
piece          i                                                        
c09n07_douceur 86       (0, 4, 1)          ()     0          0   <NA>   
               87       (5, 2, 1)          ()     1          5   <NA>   
               88      (-2, 2, 1)          ()     1         -2   <NA>   
               89   (3, 0, -3, 1)          ()     3          3   <NA>   
               90  (-4, 0, 2, -1)          ()     2         -4   <NA>   

                  alt_label bass_name  
piece          i                       
c09n07_douceur 86       NaN         E  
               87       NaN        D#  
               88       NaN         D  
               89       NaN        C#  
               90       NaN         C

#### Successive ascending semitones

In [12]:
asc = cnt(pc_ivs, 1)
asc.n.value_counts()

n
1    698
2     19
3     10
5      2
4      1
Name: count, dtype: int64

##### Storing those with three or more

In [13]:
three_asc = labels.loc[asc[asc.n > 2].ixs.sum()]
three_asc.to_csv(os.path.join(RESULTS_PATH, 'three_asc.tsv'), sep='\t')
three_asc.head(30)

mc  mn quarterbeats  duration_qb mc_onset mn_onset  \
piece             i                                                        
c03n01_prelude    26   8   7         97/4          1.0        0        0   
                  27   8   7        101/4          0.5      1/4      1/4   
                  28   8   7        103/4          0.5      3/8      3/8   
                  29   8   7        105/4          1.0      1/2      1/2   
                  30   8   7        109/4          1.0      3/4      3/4   
c03n02_allemande  52  11   8           35          1.0        0        0   
                  53  11   8           36          1.0      1/4      1/4   
                  54  11   8           37          1.0      1/2      1/2   
                  55  11   8           38          0.5      3/4      3/4   
                  56  11   8         77/2          0.5      7/8      7/8   
                  57  12   9           39          1.0        0        0   
                  58  12   9           40          0.5      1/4      1/4   
                  59  12   9         81/2          0.5      3/8      3/8   
                  60  12   9           41          0.5      1/2      1/2   
c03n03_courante   82  23  21        249/2          2.0        1        1   
                  83  24  22        253/2          1.0        0        0   
                  84  24  22        255/2          1.0      1/4      1/4   
                  85  24  22        257/2          1.0      1/2      1/2   
                  90  26  23        273/2          2.0        1        1   
                  91  27  24        277/2          1.0        0        0   
                  92  27  24        279/2          1.0      1/4      1/4   
                  93  27  24        281/2          1.0      1/2      1/2   
c06n02_allemande  23   8   7         57/2          2.0      1/2      1/2   
                  24   9   8         61/2          2.0        0        0   
                  25   9   8         65/2          1.0      1/2      1/2   
                  26   9   8         67/2          1.0      3/4      3/4   
                  27  10   9         69/2          1.0        0        0   
c06n04_air_diable 90  41  39        153/2          2.0        0        0   
                  91  42  40        157/2          2.0        0        0   
                  92  43  41        161/2          2.0        0        0   

                     timesig  staff  voice    label globalkey localkey pedal  \
piece             i                                                            
c03n01_prelude    26     4/4      1      1       I6         A        V   NaN   
                  27     4/4      1      1     ii65         A        V   NaN   
                  28     4/4      1      1    V65/V         A        V   NaN   
                  29     4/4      1      1    V(64)         A        V   NaN   
                  30     4/4      1      1        V         A        V   NaN   
c03n02_allemande  52     4/4      1      1        V         A       vi   NaN   
                  53     4/4      1      1    III+6         A       vi   NaN   
                  54     4/4      1      1       VI         A       vi   NaN   
                  55     4/4      1      1      iv6         A       vi   NaN   
                  56     4/4      1      1  V65/VII         A       vi   NaN   
                  57     4/4      1      1      VII         A       vi   NaN   
                  58     4/4      1      1       v6         A       vi   NaN   
                  59     4/4      1      1       V6         A       vi   NaN   
                  60     4/4      1      1        i         A       vi   NaN   
c03n03_courante   82     3/2      1      1    ii%65         a        i   NaN   
                  83     3/2      1      1    V65/V         a        i   NaN   
                  84     3/2      1      1        V         a        i   NaN   
                  85     3/2      1      1    ii%43         a        

##### Storing those with four or more

In [14]:
four_asc = labels.loc[asc[asc.n > 3].ixs.sum()]
four_asc.to_csv(os.path.join(RESULTS_PATH, 'four_asc.tsv'), sep='\t')
four_asc.head(30)

mc  mn quarterbeats  duration_qb mc_onset mn_onset  \
piece            i                                                        
c03n02_allemande 52  11   8           35          1.0        0        0   
                 53  11   8           36          1.0      1/4      1/4   
                 54  11   8           37          1.0      1/2      1/2   
                 55  11   8           38          0.5      3/4      3/4   
                 56  11   8         77/2          0.5      7/8      7/8   
                 57  12   9           39          1.0        0        0   
                 58  12   9           40          0.5      1/4      1/4   
                 59  12   9         81/2          0.5      3/8      3/8   
                 60  12   9           41          0.5      1/2      1/2   
c09n06_Sarabande 29  17  16           47          1.0      1/2      1/2   
                 30  18  17           48          3.0        0        0   
                 31  19  18           51          3.0        0        0   
                 32  20  19           54          3.0        0        0   
                 33  21  20           57          1.5        0        0   
                 34  21  20        117/2          0.5      3/8      3/8   
c11n02_allemande 81  18  16        131/2          0.5      5/8      5/8   
                 82  18  16           66          1.0      3/4      3/4   
                 83  19  17           67          0.5        0        0   
                 84  19  17        135/2          0.5      1/8      1/8   
                 85  19  17           68          0.5      1/4      1/4   

                    timesig  staff  voice     label globalkey localkey pedal  \
piece            i                                                             
c03n02_allemande 52     4/4      1      1         V         A       vi   NaN   
                 53     4/4      1      1     III+6         A       vi   NaN   
                 54     4/4      1      1        VI         A       vi   NaN   
                 55     4/4      1      1       iv6         A       vi   NaN   
                 56     4/4      1      1   V65/VII         A       vi   NaN   
                 57     4/4      1      1       VII         A       vi   NaN   
                 58     4/4      1      1        v6         A       vi   NaN   
                 59     4/4      1      1        V6         A       vi   NaN   
                 60     4/4      1      1         i         A       vi   NaN   
c09n06_Sarabande 29     3/4      1      1       I\\         e      III   NaN   
                 30     3/4      1      1  i.V65/iv         e        i   NaN   
                 31     3/4      1      1        IV         e        i   NaN   
                 32     3/4      1      1     V65/V         e        i   NaN   
                 33     3/4      1      1         V         e        i   NaN   
                 34     3/4      1      1     ii%43         e        i   NaN   
c11n02_allemande 81     4/4      1      1       i64         c        i   NaN   
                 82     4/4      1      1       iv6         c        i   NaN   
                 83     4/4      1      1   V65/VII         c        i   NaN   
                 84     4/4      1      1       VII         c        i   NaN   
                 85     4/4      1      1       V65         c        i   NaN   

                       chord numeral form figbass changes relativeroot  \
piece            i                                                       
c03n02_allemande 52        V       V  NaN     NaN     NaN          NaN   
                 53    III+6     III    +       6     NaN          NaN   
                 54       VI      VI  NaN     NaN     NaN          NaN   
                 55      iv6      iv  NaN       6     NaN          NaN   
                 56  V65/VII       V  NaN      65     NaN          VII   
                 57      VII     VII  NaN     NaN     NaN          NaN   
                 

### Filtering for particular progressions with length >= 3
Finding only direct successors

In [15]:
def filtr(df, query, column='chord'):
    vals = df[column].to_list()
    n_grams = [t for t in zip(*(vals[i:] for i in range(len(query))))]
    if isinstance(query[0], str):
        lengths = [len(q) for q in query]
        n_grams = [tuple(e[:l] for e,l  in zip(t, lengths)) for t in n_grams]
    return query in n_grams

def show(df, query, column='chord'):
    selector = df.groupby(level=0).apply(filtr, query, column)
    return df[selector[df.index.get_level_values(0)].values]

#### Descending

In [16]:
descending = pd.concat([labels.loc[ix_seq] for ix_seq in desc[desc.n > 2].ixs.values], keys=range((desc.n > 2).sum()))
descending

mc  mn quarterbeats  duration_qb mc_onset mn_onset  \
  piece             i                                                        
0 c02n01_prelude    23  13  11           35         2.00      1/4      1/4   
                    24  14  12           37         2.00        0        0   
                    25  14  12           39         1.00      1/2      1/2   
                    26  15  13           40         1.00        0        0   
                    27  15  13           41         1.00      1/4      1/4   
1 c02n01_prelude    53  25  23           70         2.00        0        0   
                    54  25  23           72         1.00      1/2      1/2   
                    55  26  24           73         2.00        0        0   
                    56  26  24           75         1.00      1/2      1/2   
                    57  27  25           76         1.00        0        0   
                    58  27  25           77         1.00      1/4      1/4   
2 c02n03_air_tendre 34  17  17           48         2.00        0        0   
                    35  17  17           50         1.00      1/2      1/2   
                    36  18  18           51         1.00        0        0   
                    37  18  18           52         1.00      1/4      1/4   
                    38  18  18           53         1.00      1/2      1/2   
                    39  19  19           54         1.00        0        0   
3 c05n02_allemande  37   8   7           30         0.50      7/8      7/8   
                    38   9   8         61/2         2.00        0        0   
                    39   9   8         65/2         1.00      1/2      1/2   
                    40   9   8         67/2         1.00      3/4      3/4   
                    41  10   9         69/2         1.00        0        0   
                    42  10   9         71/2         1.00      1/4      1/4   
4 c09n07_douceur    86  39  37           56         0.25     5/16     5/16   
                    87  40  38        225/4         1.00        0        0   
                    88  40  38        229/4         0.50      1/4      1/4   
                    89  41  39        231/4         1.50        0        0   
                    90  42  40        237/4         0.50        0        0   

                       timesig  staff  voice   label globalkey localkey pedal  \
  piece             i                                                           
0 c02n01_prelude    23     3/4      1      1   V.I\\         D        V   NaN   
                    24     3/4      1      1      V6         D        V   NaN   
                    25     3/4      1      1  iii%43         D        V   NaN   
                    26     3/4      1      1  IV6(2)         D        V   NaN   
                    27     3/4      1      1     IV6         D        V   NaN   
1 c02n01_prelude    53     3/4      1      1     I\\         D        I   NaN   
                    54     3/4      1      1     vi6         D        I   NaN   
                    55     3/4      1      1   vii%7         D        I   NaN   
                    56     3/4      1      1  iii%43         D        I   NaN   
                    57     3/4      1      1     vi7         D        I   NaN   
                    58     3/4      1      1     IV6         D        I   NaN   
2 c02n03_air_tendre 34     3/4      1      1   III.I         d      III   NaN   
                    35     3/4      1      1    V2/V         d      III   NaN   
                    36     3/4      1      1      V6         d      III   NaN   
                    37     3/4      1      1   IV/IV         d      III   NaN   
                    38     3/4      1      1   V2/IV         d      III   NaN   
                    39     3/4      1      1     IV6         d      III   NaN   
3 c05n02_allemande  37     2/2      1      1       I         F        V   NaN   
                    38     2/2      1      1      V6         F 

##### Looking for `Ger i64`

In [17]:
show(descending, ('Ger', 'i64'))

,,,mc,mn,quarterbeats,duration_qb,mc_onset,mn_onset,timesig,staff,voice,label,globalkey,localkey,pedal,chord,numeral,form,figbass,changes,relativeroot,cadence,phraseend,chord_type,globalkey_is_minor,localkey_is_minor,chord_tones,added_tones,root,bass_note,volta,alt_label,bass_name
,piece,i,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


##### `i64`

In [18]:
show(descending, ('i64',))

,,,mc,mn,quarterbeats,duration_qb,mc_onset,mn_onset,timesig,staff,voice,label,globalkey,localkey,pedal,chord,numeral,form,figbass,changes,relativeroot,cadence,phraseend,chord_type,globalkey_is_minor,localkey_is_minor,chord_tones,added_tones,root,bass_note,volta,alt_label,bass_name
,piece,i,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


##### `Ger V(64)`

In [19]:
show(descending, ('Ger', 'V(64'))

,,,mc,mn,quarterbeats,duration_qb,mc_onset,mn_onset,timesig,staff,voice,label,globalkey,localkey,pedal,chord,numeral,form,figbass,changes,relativeroot,cadence,phraseend,chord_type,globalkey_is_minor,localkey_is_minor,chord_tones,added_tones,root,bass_note,volta,alt_label,bass_name
,piece,i,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


##### Bass degrees `b6 5 ###4`

In [20]:
show(descending, (-4, 1, 6), 'bass_note')

,,,mc,mn,quarterbeats,duration_qb,mc_onset,mn_onset,timesig,staff,voice,label,globalkey,localkey,pedal,chord,numeral,form,figbass,changes,relativeroot,cadence,phraseend,chord_type,globalkey_is_minor,localkey_is_minor,chord_tones,added_tones,root,bass_note,volta,alt_label,bass_name
,piece,i,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


#### Ascending

In [21]:
ascending = pd.concat([labels.loc[ix_seq] for ix_seq in asc[asc.n > 2].ixs.values], keys=range((asc.n > 2).sum()))
ascending = ascending[ascending.label != '@none']
ascending

mc  mn quarterbeats  duration_qb mc_onset mn_onset  \
   piece             i                                                         
0  c03n01_prelude    26    8   7         97/4         1.00        0        0   
                     27    8   7        101/4         0.50      1/4      1/4   
                     28    8   7        103/4         0.50      3/8      3/8   
                     29    8   7        105/4         1.00      1/2      1/2   
                     30    8   7        109/4         1.00      3/4      3/4   
1  c03n02_allemande  52   11   8           35         1.00        0        0   
                     53   11   8           36         1.00      1/4      1/4   
                     54   11   8           37         1.00      1/2      1/2   
                     55   11   8           38         0.50      3/4      3/4   
                     56   11   8         77/2         0.50      7/8      7/8   
                     57   12   9           39         1.00        0        0   
                     58   12   9           40         0.50      1/4      1/4   
                     59   12   9         81/2         0.50      3/8      3/8   
                     60   12   9           41         0.50      1/2      1/2   
2  c03n03_courante   82   23  21        249/2         2.00        1        1   
                     83   24  22        253/2         1.00        0        0   
                     84   24  22        255/2         1.00      1/4      1/4   
                     85   24  22        257/2         1.00      1/2      1/2   
3  c03n03_courante   90   26  23        273/2         2.00        1        1   
                     91   27  24        277/2         1.00        0        0   
                     92   27  24        279/2         1.00      1/4      1/4   
                     93   27  24        281/2         1.00      1/2      1/2   
4  c06n02_allemande  23    8   7         57/2         2.00      1/2      1/2   
                     24    9   8         61/2         2.00        0        0   
                     25    9   8         65/2         1.00      1/2      1/2   
                     26    9   8         67/2         1.00      3/4      3/4   
                     27   10   9         69/2         1.00        0        0   
5  c06n04_air_diable 90   41  39        153/2         2.00        0        0   
                     91   42  40        157/2         2.00        0        0   
                     92   43  41        161/2         2.00        0        0   
                     93   44  42        165/2         1.00        0        0   
6  c08n01_ouverture  132  84  82        521/2         3.00        0        0   
                     133  85  83        527/2         3.00        0        0   
                     134  86  84        533/2         3.00        0        0   
                     135  87  85        539/2         3.00        0        0   
7  c09n06_Sarabande  29   17  16           47         1.00      1/2      1/2   
                     30   18  17           48         3.00        0        0   
                     31   19  18           51         3.00        0        0   
                     32   20  19           54         3.00        0        0   
                     33   21  20           57         1.50        0        0   
                     34   21  20        117/2         0.50      3/8      3/8   
8  c09n07_douceur    79   37  35        207/4         0.50        0        0   
                     80   37  35        209/4         0.75      1/8      1/8   
                     81   37  35           53         0.25     5/16     5/16   
                     82   38  36        213/4         0.50        0        0   
                     83   38  36        215/4         1.00      1/8      1/8   
                     84   39  37        219/4         0.50        0        0   
9  c11n02_allemande  81   18  16        131/2         0.50      5/8      5/8   
                     82   

##### `i64 Ger`

In [22]:
show(ascending, ('i64', 'Ger'))

,,,mc,mn,quarterbeats,duration_qb,mc_onset,mn_onset,timesig,staff,voice,label,globalkey,localkey,pedal,chord,numeral,form,figbass,changes,relativeroot,cadence,phraseend,chord_type,globalkey_is_minor,localkey_is_minor,chord_tones,added_tones,root,bass_note,volta,alt_label,bass_name
,piece,i,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


##### `i64`

In [23]:
show(ascending, ('i64',))

mc  mn quarterbeats  duration_qb mc_onset mn_onset  \
  piece            i                                                        
9 c11n02_allemande 81  18  16        131/2          0.5      5/8      5/8   
                   82  18  16           66          1.0      3/4      3/4   
                   83  19  17           67          0.5        0        0   
                   84  19  17        135/2          0.5      1/8      1/8   
                   85  19  17           68          0.5      1/4      1/4   

                      timesig  staff  voice    label globalkey localkey pedal  \
  piece            i                                                            
9 c11n02_allemande 81     4/4      1      1      i64         c        i   NaN   
                   82     4/4      1      1      iv6         c        i   NaN   
                   83     4/4      1      1  V65/VII         c        i   NaN   
                   84     4/4      1      1      VII         c        i   NaN   
                   85     4/4      1      1      V65         c        i   NaN   

                         chord numeral form figbass changes relativeroot  \
  piece            i                                                       
9 c11n02_allemande 81      i64       i  NaN      64     NaN          NaN   
                   82      iv6      iv  NaN       6     NaN          NaN   
                   83  V65/VII       V  NaN      65     NaN          VII   
                   84      VII     VII  NaN     NaN     NaN          NaN   
                   85      V65       V  NaN      65     NaN          NaN   

                      cadence phraseend chord_type  globalkey_is_minor  \
  piece            i                                                     
9 c11n02_allemande 81     NaN       NaN          m                True   
                   82     NaN       NaN          m                True   
                   83     NaN       NaN        Mm7                True   
                   84     NaN       NaN          M                True   
                   85     NaN       NaN        Mm7                True   

                       localkey_is_minor     chord_tones added_tones  root  \
  piece            i                                                         
9 c11n02_allemande 81               True      (1, 0, -3)          ()     0   
                   82               True     (-4, 0, -1)          ()    -1   
                   83               True  (3, 0, -3, -1)          ()    -1   
                   84               True     (-2, 2, -1)          ()    -2   
                   85               True   (5, 2, -1, 1)          ()     1   

                       bass_note  volta alt_label bass_name  
  piece            i                                         
9 c11n02_allemande 81          1   <NA>       NaN         G  
                   82         -4   <NA>       NaN        Ab  
                   83          3   <NA>       NaN         A  
                   84         -2   <NA>       NaN        Bb  
                   85          5   <NA>       NaN         B

##### `V(64) Ger`

In [24]:
show(ascending, ('V(64)', 'Ger'))

,,,mc,mn,quarterbeats,duration_qb,mc_onset,mn_onset,timesig,staff,voice,label,globalkey,localkey,pedal,chord,numeral,form,figbass,changes,relativeroot,cadence,phraseend,chord_type,globalkey_is_minor,localkey_is_minor,chord_tones,added_tones,root,bass_note,volta,alt_label,bass_name
,piece,i,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


##### Bass degrees `###4 5 b6`

In [25]:
show(ascending, (6, 1, -4), 'bass_note')

mc  mn quarterbeats  duration_qb mc_onset mn_onset  \
  piece            i                                                        
2 c03n03_courante  82  23  21        249/2          2.0        1        1   
                   83  24  22        253/2          1.0        0        0   
                   84  24  22        255/2          1.0      1/4      1/4   
                   85  24  22        257/2          1.0      1/2      1/2   
3 c03n03_courante  90  26  23        273/2          2.0        1        1   
                   91  27  24        277/2          1.0        0        0   
                   92  27  24        279/2          1.0      1/4      1/4   
                   93  27  24        281/2          1.0      1/2      1/2   
7 c09n06_Sarabande 29  17  16           47          1.0      1/2      1/2   
                   30  18  17           48          3.0        0        0   
                   31  19  18           51          3.0        0        0   
                   32  20  19           54          3.0        0        0   
                   33  21  20           57          1.5        0        0   
                   34  21  20        117/2          0.5      3/8      3/8   

                      timesig  staff  voice     label globalkey localkey  \
  piece            i                                                       
2 c03n03_courante  82     3/2      1      1     ii%65         a        i   
                   83     3/2      1      1     V65/V         a        i   
                   84     3/2      1      1         V         a        i   
                   85     3/2      1      1     ii%43         a        i   
3 c03n03_courante  90     3/2      1      1     II%65         a        i   
                   91     3/2      1      1     V65/V         a        i   
                   92     3/2      1      1         V         a        i   
                   93     3/2      1      1     ii%43         a        i   
7 c09n06_Sarabande 29     3/4      1      1       I\\         e      III   
                   30     3/4      1      1  i.V65/iv         e        i   
                   31     3/4      1      1        IV         e        i   
                   32     3/4      1      1     V65/V         e        i   
                   33     3/4      1      1         V         e        i   
                   34     3/4      1      1     ii%43         e        i   

                      pedal   chord numeral form figbass changes relativeroot  \
  piece            i                                                            
2 c03n03_courante  82   NaN   ii%65      ii    %      65     NaN          NaN   
                   83   NaN   V65/V       V  NaN      65     NaN            V   
                   84   NaN       V       V  NaN     NaN     NaN          NaN   
                   85   NaN   ii%43      ii    %      43     NaN          NaN   
3 c03n03_courante  90   NaN   II%65      II    %      65     NaN          NaN   
                   91   NaN   V65/V       V  NaN      65     NaN            V   
                   92   NaN       V       V  NaN     NaN     NaN          NaN   
                   93   NaN   ii%43      ii    %      43     NaN          NaN   
7 c09n06_Sarabande 29   NaN       I       I  NaN     NaN     NaN          NaN   
                   30   NaN  V65/iv       V  NaN      65     NaN           iv   
                   31   NaN      IV      IV  NaN     NaN     NaN          NaN   
                   32   NaN   V65/V       V  NaN      65     NaN            V   
                   33   NaN       V       V  NaN     NaN     NaN          NaN   
                   34   NaN   ii%43      ii    %      43     NaN          NaN   

                      cadence phraseend chord_type  globalkey_is_minor  \
  piece            i                                                     
2 c03n03_courante  82     NaN       NaN         %7                True   
                   83     NaN       NaN        Mm7    